# News Source Classification

In [1]:
import pandas as pd
from pprint import pprint as pp
from collections import Counter
import matplotlib.pyplot as plt

In [2]:
d = pd.read_csv('/root/files/sql_result.csv')
print(d.columns)

Index(['id', 'author', 'source', 'content', 'feature', 'title', 'url'], dtype='object')


# Only need the x:content and y:source
## first check the label distribution

In [3]:
%matplotlib widget
import matplotlib as  mpl
source_cnt = d['source'].value_counts()
s_cnt = dict(source_cnt)
s_cnt = sorted(s_cnt.items(),key=lambda x:x[1],reverse=True)[:10]
sc = [i[0] for i in s_cnt]
cnt = [i[1] for i in s_cnt]
plt.bar(sc,cnt)
plt.show()

FigureCanvasNbAgg()

## only take the top 10 lables

In [4]:
content = d.content.tolist()
sc_labels_t = {s:i for i,s in enumerate(sc)}
total = len(d)
fd = d[d['source'].isin(set(sc))]
pp(f'delete {total - len(content)}')
pp(sc_labels_t)
fd.head()

'delete 0'
{'中国台湾网': 7,
 '中国新闻网': 3,
 '中国证券报?中证网': 2,
 '南方日报第01版': 6,
 '参考消息网': 4,
 '央广网': 8,
 '微博': 1,
 '新华社': 0,
 '新华网': 9,
 '环球网': 5}


,id,author,source,content,feature,title,url
3,89614,NaN,新华社,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\n,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",葡森林火灾造成至少62人死亡 政府宣布进入紧急状态（组图）,http://world.huanqiu.com/hot/2017-06/10866126....
5,89612,张怡,中国证券报?中证网,受到A股被纳入MSCI指数的利好消息刺激，A股市场从周三开始再度上演龙马行情，周四上午金...,"{""type"":""市场"",""site"":""中证网"",""commentNum"":""0"",""jo...",金融股一枝独秀 配置价值犹存,http://www.cs.com.cn/gppd/201706/t20170623_533...
7,89610,申玉彬 整理,中国证券报?中证网,沙漠雄鹰：震荡有利消化套牢筹码\n 周四开盘上证50在银行券商大蓝筹带动下一度涨近2%...,"{""type"":""市场"",""site"":""中证网"",""commentNum"":""0"",""jo...",博友早评：震荡有利消化套牢筹码,http://www.cs.com.cn/gppd/201706/t20170623_533...
9,89608,吴瞬,中国证券报?中证网,6月21日，A股纳入MSCI指数尘埃落定，但当天被寄予厚望的券商股并未扛起反弹大旗。22...,"{""type"":""市场"",""site"":""中证网"",""commentNum"":""0"",""jo...",纳入MSCI指数 A股长期配置价值提升,http://www.cs.com.cn/gppd/201706/t20170623_533...
12,89605,NaN,中国新闻网,中新网6月19日电 据外媒报道，美国底特律一名男子1976年因为一根头发被定谋杀罪，监禁41...,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",因为犯罪现场的一根头发，他坐冤狱41年后终获释,http://world.huanqiu.com/hot/2017-06/10866136....


In [5]:
content = fd.content.tolist()
sc_labels = fd.source.apply(lambda x:sc_labels_t[x]).tolist()

# tools

In [6]:
import jieba
def cut(s):
    return list(jieba.cut(s))

# Design feature by content

In [7]:
import numpy as np
content = [str(i).replace('\n','').replace('\u3000','') for i in content]

## TF-IDF

In [8]:
%%time
# jieba.enable_parallel(10)
cut_content = [cut(i) for i in content]

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.855 seconds.
Prefix dict has been built succesfully.


CPU times: user 2min 43s, sys: 1.58 s, total: 2min 44s
Wall time: 2min 44s


In [9]:
from collections import Counter
cnt = Counter()
for c in cut_content: cnt.update(c)
print(len(cnt))
# w2i = {k:v for v,k in enumerate(words)}
# i2w = {v:k for k,v in w2i.items()}

216071


filter the tokens by min_count

In [10]:
min_count = 10
tokens = set(filter(lambda x:x[1] > min_count,dict(cnt).items()))
tokens = [i[0] for i in tokens]
print(len(tokens))
w2i = {k:v for v,k in enumerate(tokens)}
i2w = {v:k for k,v in w2i.items()}

56000


In [ ]:
from collections import Counter
def calculate_term_frequency(c):
    '''
    @c : cut sentence.
    @output : tf
    '''
    cnt = Counter(c)
    tf = np.zeros((len(tokens)))
    for k,v in dict(cnt).items():
        if k not in tokens :
            continue
        tf[w2i[k]] = v / len(c)
    return tf
TF = []
for c in cut_content:
    TF.append(calculate_term_frequency(c))
TF = np.array(TF)
TF.shape

In [2]:
TF.shape

NameError: name 'TF' is not defined

In [1]:
sc_labels = fd.source.apply(lambda x:sc_labels_t[x]).tolist()
print(len(sc_labels))

NameError: name 'fd' is not defined

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(TF,sc_labels)
print(len(x_train))
lr = LogisticRegression(n_jobs=64)
lr.fit(x_train,y_train)
lr.score(x_test,y_test)

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/root/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/root/anaconda3/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/root/anaconda3/lib/python3.6/multiprocessing/pool.py", line 405, in _handle_workers
    pool._maintain_pool()
  File "/root/anaconda3/lib/python3.6/multiprocessing/pool.py", line 246, in _maintain_pool
    self._repopulate_pool()
  File "/root/anaconda3/lib/python3.6/multiprocessing/pool.py", line 239, in _repopulate_pool
    w.start()
  File "/root/anaconda3/lib/python3.6/multiprocessing/process.py", line 105, in start
    self._popen = self._Popen(self)
  File "/root/anaconda3/lib/python3.6/multiprocessing/context.py", line 277, in _Popen
    return Popen(process_obj)
  File "/root/anaconda3/lib/python3.6/multiprocessing/popen_fork.py", line 19, in __init__
    self._launch(process_obj)
  File "/root/

In [ ]:
print(lr.score(x_test,y_test))